In [1]:
from configs import models, env
from backend.llm.api import load_llm_langchain

config_loaded = {"model_config": models, "env": env}
llm = load_llm_langchain(source='ollama', model_name='LLaMA-3', config=config_loaded)

[LLM Loader] Successfully initialized model 'llama3' from 'ollama'.


In [2]:
from backend.agents.api import (
    run_symptom_checker,
    run_ehr_summarizer,
    run_literature_qa,
    run_drug_interactions,
    run_treatment_plan,
)

In [7]:
def symptom_node(state):
    symptoms = state.get("symptoms", "")
    if symptoms:
        state["diagnosis"] = run_symptom_checker(symptoms, llm)
    return state


def ehr_node(state):
    notes = state.get("ehr_text", "")
    if notes:
        state["summary"] = run_ehr_summarizer(notes, llm)
    return state


def literature_node(state):
    question = state.get("question", "")
    context = state.get("summary", "") or state.get("ehr_text", "")
    if question:
        state["literature_answer"] = run_literature_qa(question, llm, context)
    return state


def drug_node(state):
    meds = state.get("medications", [])
    if isinstance(meds, str):
        meds = [m.strip() for m in meds.split(",")]
    elif not isinstance(meds, list):
        meds = []

    patient_data = state.get("patient_profile", "")
    state["interaction_report"] = run_drug_interactions(meds, llm, patient_data)
    return state


def treatment_node(state):
    profile = state.get("patient_profile", {})
    if isinstance(profile, dict) and profile:
        state["treatment_plan"] = run_treatment_plan(profile, llm)
    return state


In [4]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional, List

# Step 1: Define the schema
class AgentState(TypedDict):
    symptoms: Optional[str]
    ehr_text: Optional[str]
    meds: Optional[List[str]]
    question: Optional[str]
    diagnosis: Optional[str]
    summary: Optional[str]
    answer: Optional[str]
    plan: Optional[str]
    drug_warnings: Optional[str]

In [8]:
graph = StateGraph(state_schema=AgentState)


graph.add_node("symptom_checker", symptom_node)
graph.add_node("ehr_summarizer", ehr_node)
graph.add_node("literature_qa", literature_node)
graph.add_node("drug_checker", drug_node)
graph.add_node("treatment_planner", treatment_node)

# Add edges (simple linear for now, can be conditional later)
graph.set_entry_point("symptom_checker")
graph.add_edge("symptom_checker", "ehr_summarizer")
graph.add_edge("ehr_summarizer", "literature_qa")
graph.add_edge("literature_qa", "drug_checker")
graph.add_edge("drug_checker", "treatment_planner")
graph.add_edge("treatment_planner", END)

graph_executor = graph.compile()

In [9]:
initial_state = {
    "symptoms": "shortness of breath, chest pain, fatigue",
    "ehr_text": "Patient has a history of hypertension and presents with elevated troponins...",
    "question": "What is the current recommendation for NSTEMI management in elderly patients?",
    "medications": ["Aspirin", "Warfarin"],
    "patient_profile": {
        "diagnosis": "NSTEMI",
        "age": 72,
        "sex": "Male",
        "comorbidities": ["Hypertension", "Atrial Fibrillation"]
    }
}
for state in graph_executor.stream(initial_state):
    # 'state' will be a dictionary representing the current state of the graph
    # You might want to print just the relevant part, e.g., the latest message or the 'output'
    print(f"Current state: {state}")
    # If your 'AgentState' has an 'output' key that holds the final response:
    # if "output" in state:
    #    print(f"Output fragment: {state['output']}")

print("\n--- End of Streaming GraphExecutor Output ---")

Current state: {'symptom_checker': {'symptoms': 'shortness of breath, chest pain, fatigue', 'ehr_text': 'Patient has a history of hypertension and presents with elevated troponins...', 'question': 'What is the current recommendation for NSTEMI management in elderly patients?', 'diagnosis': AIMessage(content="Based on the symptoms of shortness of breath, chest pain, and fatigue, here is a ranked list of likely diagnoses:\n\n1. **Pulmonary Embolism** (PE): A blood clot in the lungs that can cause sudden onset of shortness of breath, chest pain, and fatigue.\n2. **Acute Coronary Syndrome** (ACS): A blockage or narrowing of the coronary arteries that can cause chest pain, shortness of breath, and fatigue.\n3. **Pneumonia**: An infection in the lungs that can cause coughing, fever, and difficulty breathing.\n4. **Chronic Obstructive Pulmonary Disease** (COPD): A progressive lung disease that can cause chronic shortness of breath, chest pain, and fatigue.\n\nUrgency level: **High**\n\nRecomm

In [23]:
for key, value in state['treatment_planner'].items():
    print(f"\n📌 {key.upper()}:\n{value.content if isinstance(value, str) == False else value}\n")


📌 SYMPTOMS:
shortness of breath, chest pain, fatigue


📌 EHR_TEXT:
Patient has a history of hypertension and presents with elevated troponins...


📌 QUESTION:
What is the current recommendation for NSTEMI management in elderly patients?


📌 DIAGNOSIS:
Based on the symptoms of shortness of breath, chest pain, and fatigue, here is a ranked list of likely diagnoses:

1. **Pulmonary Embolism** (PE): A blood clot in the lungs that can cause sudden onset of shortness of breath, chest pain, and fatigue.
2. **Acute Coronary Syndrome** (ACS): A blockage or narrowing of the coronary arteries that can cause chest pain, shortness of breath, and fatigue.
3. **Pneumonia**: An infection in the lungs that can cause coughing, fever, and difficulty breathing.
4. **Chronic Obstructive Pulmonary Disease** (COPD): A progressive lung disease that can cause chronic shortness of breath, chest pain, and fatigue.

Urgency level: **High**

Recommended next steps:

1. **Immediate evaluation**: The patient should

In [ ]:
initial_state = {
    "symptoms": "shortness of breath, chest pain, fatigue",
    "ehr_text": "Patient has a history of hypertension and presents with elevated troponins...",
    "question": "What is the current recommendation for NSTEMI management in elderly patients?",
    "medications": ["Aspirin", "Warfarin"],
    "patient_profile": {
        "diagnosis": "NSTEMI",
        "age": 72,
        "sex": "Male",
        "comorbidities": ["Hypertension", "Atrial Fibrillation"]
    }
}

final_state = graph_executor.invoke(initial_state)

for key, value in final_state.items():
    print(f"\n📌 {key.upper()}:\n{value.content if isinstance(value, str) == False else value}\n")


📌 SYMPTOMS:
shortness of breath, chest pain, fatigue


📌 EHR_TEXT:
Patient has a history of hypertension and presents with elevated troponins...


📌 QUESTION:
What is the current recommendation for NSTEMI management in elderly patients?


📌 DIAGNOSIS:
Based on the symptoms of shortness of breath, chest pain, and fatigue, here is a ranked list of likely diagnoses:

1. **Pulmonary Embolism** (PE): A blood clot in the lungs that can cause sudden onset of shortness of breath, chest pain, and fatigue.
2. **Acute Coronary Syndrome** (ACS): A blockage or narrowing of the coronary arteries that can cause chest pain, shortness of breath, and fatigue.
3. **Pneumonia**: An infection in the lungs that can cause coughing, fever, and difficulty breathing.
4. **Chronic Obstructive Pulmonary Disease** (COPD): A progressive lung disease that can cause chronic shortness of breath, chest pain, and fatigue.

Urgency level: **High**

Recommended next steps:

1. **Immediate evaluation**: The patient should